In [1]:
from dotenv import load_dotenv

import openai
import os
load_dotenv()

import logging
import sys
import os

import numexpr as ne

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
openai.api_key = os.getenv('OPENAI_API_KEY')


os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '2'

In [2]:
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)
from llama_index.prompts import PromptTemplate

try:
    storage_context = StorageContext.from_defaults(persist_dir='../assets/test_store/')
    index = load_index_from_storage(storage_context)
    print('loading from disk')
except:
    documents = SimpleDirectoryReader('../assets/AndrewHuberman/medical').load_data()
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir='../storage/cache/andrew/medical/')
    print('persisting to disk')

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
loading from disk


In [3]:
openai.log = "debug"

text_qa_template_str = (
    "Bạn là trợ lý của Andrew Huberman, người có thể đọc các ghi chú podcast của Andrew Huberman.\n"
    "Luôn trả lời truy vấn chỉ bằng cách sử dụng thông tin ngữ cảnh được cung cấp, "
    "chứ không phải kiến ​​thức sẵn có.\n"
    "Một số quy tắc cần tuân theo:\n"
    "1. Không bao giờ tham khảo trực tiếp bối cảnh nhất định trong câu trả lời của bạn.\n"
    "2. Tránh những câu như 'Dựa trên ngữ cảnh, ...' hoặc "
    "'Thông tin ngữ cảnh ...' hoặc bất cứ điều gì cùng"
    "những dòng đó."
    "Thông tin bối cảnh dưới đây.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Trả lời câu hỏi:{query_str}\n"
)

text_qa_template = PromptTemplate(text_qa_template_str)

In [4]:
from llama_index.postprocessor import SentenceTransformerRerank


rerank = SentenceTransformerRerank(
    model="sentence-transformers/all-mpnet-base-v2",
    top_n=5
)

Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: cpu
Use pytorch device: cpu


In [18]:
from llama_index import ServiceContext
service_context = ServiceContext.from_defaults(
    chunk_size=50,
    chunk_overlap=3
)

In [19]:
from time import time

query_engine = index.as_query_engine(
    similarity_top_k=10,
    # node_postprocessors=[rerank],
    service_context=service_context,
    text_qa_template = text_qa_template,
)

In [20]:
now = time()
response = query_engine.query(
    "ung thư dạ dày là bệnh gì?",
)
print(f"Elapsed: {round(time() - now, 2)}s")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Elapsed: 10.38s


In [21]:
print(response)

Ung thư dạ dày là một căn bệnh nguy hiểm trong hệ tiêu hóa, có tỷ lệ tử vong cao và là một trong những loại ung thư phổ biến nhất trên thế giới.


In [22]:
for i in response.source_nodes:
    print('score: ', i.score)
    print('text: ', i.text)
    print('='*100)

score:  0.8744441134637128
text:  Đặc biệt, nếu phát hiện muộn khi các tế bào ung thư đã di căn thì vô phương cứu chữa. Vì vậy, người dân nên nâng cao việc phòng chống bệnh ung thư dạ dày ngay từ sớm bằng cách.
  Duy trì chế độ sinh hoạt, luyện tập, nghỉ ngơi khoa học.
 Duy trì chế độ dinh dưỡng lành mạnh với các thực phẩm giàu vitamin và chất xơ.
 Hạn chế ăn các thực phẩm giàu nitric và amin thứ cấp như dưa muối, cà muối, đồ ăn lên men, thịt hun khói, đồ nướng. Bởi vì khi đi vào dạ dày, các chất này sẽ kết hợp thành độc tố gây nguy cơ ung thư.
 Không hút thuốc lá, uống rượu, bia hoặc sử dụng chất kích thích.
 Hạn chế đồ ăn công nghiệp và nước ngọt đóng chai.
 Chủ động tầm soát ung thư dạ dày sớm đối với các trường hợp có yếu tố nguy cơ.
   Mặc dù độ tuổi thường gặp 
<i>ung thư dạ dày
</i> là từ 50 tuổi trở lên, nhưng những năm gần đây, căn bệnh này đang có xu hướng trẻ hóa và phổ biến cả ở những người dưới 40 tuổi. Tuy nhiên, nếu được phát hiện sớm từ giai đoạn khởi phát thì cơ hội đi